In [7]:
import json
import pandas as pd
import time
from pubsub.subpub_emulator.emulator.publisher import pub_msg
from pubsub.subpub_emulator.emulator.db import get_db
from sqlalchemy import create_engine

In [2]:
USER = 'ezoji'
PASSWORD = '1qaz!QAZ'
HOST = 'localhost'
PORT = '3306'
NAME = 'pubsub'

In [3]:
con_string = f'mysql://{USER}:{PASSWORD}@{HOST}'
engine = create_engine(con_string)

In [4]:
devices = pd.read_sql('SELECT * FROM pubsub.subpub_emulator_device', engine)

In [5]:
devices

,id,name,device_id,latitude,longitude
0,1,device 1,E2280DB7-124F-4895-8BB6-66D5E2B0A686,45.36,36.50
1,2,device 2,F366203D-A414-43A8-8E76-50DF0282F5B0,25.30,36.98
2,3,device 3,1C71C0C0-0A98-42CF-8351-B4A16E569FF0,56.60,67.90


In [33]:
for i in range(len(devices)):
    dev = devices.iloc[i]
    _id = dev['id']
    name = dev['name']
    device_id = dev['device_id']
    latitude = dev['latitude']
    longitude = dev['longitude']
    
    for i in range(500):
        msg = pub_msg(deviceId=device_id, latitude=latitude, longitude=longitude)
                
        time.sleep(2)
        
        #break

Temperature: 21 at Time: 1630829902


KeyboardInterrupt: 

In [9]:
from pubsub.subpub_emulator.emulator.subscriber import sub_msg

In [27]:
sub_msg()

{'type': 'subscribe', 'pattern': None, 'channel': b'Temperature', 'data': 1}

In [26]:
len(sub_msg())


TypeError: object of type 'NoneType' has no len()

In [34]:
while True:
    msg = sub_msg()
    if not msg:
        break
    data = json.loads(msg)
    ts = data['data']['time']
    temperature = data['data']['temperature']
    
    query_insert = '''insert into pubsub.subpub_emulator_temperature
                      (device_id, time, temperature) values ({}, {}, {})
                    '''.format(_id, ts, temperature)
            
    with engine.connect() as con:
        con.execute(query_insert)
    
    print(f'device: {name} -- time: {ts} -- temperature: {temperature}')

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [35]:
msg

{'type': 'subscribe', 'pattern': None, 'channel': b'Temperature', 'data': 1}